In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bisect
import copy
import datetime
import enum
import json
import math
import pathlib

In [ ]:
import fumbblapi
import roman
import srdata
import srpages
import srpoints
import srreport
import srschedule
import srslot
import srteam
import srtime
import srtournament

In [ ]:
tournamenturlfs = 'https://fumbbl.com/p/group?op=view&at=1&group={groupId}&p=tournaments&show={tournamentId}&showallrounds=1'

In [ ]:
sr_data_path = pathlib.Path('../sr-data')
sr_pages_path = pathlib.Path('../sr-pages')

In [ ]:
TI = srdata.TournamentIdx
srdata.load(sr_data_path)
srtime.regen_fumbblyears()

In [ ]:
curr_weeknr = srtime.current_weeknr()
weeknr_report_number = {w: r for r, w in enumerate(srtime.report_weeknrs(), 1)}
fumbblyears = sorted(range(1, srtime.weeknr_fumbblyear(max(weeknr_report_number)) + 1))

In [ ]:
weeknr = 670

## 1.

For a given Report, check which tournaments are included from the [Tournaments|SR-Tournaments] list. The Monday date of the Report must be greater than or equal to the tournament's ''Enter Date'' and less than the ''Exit Date''.

In [ ]:
tournaments = srtournament.weeknr_tournaments(weeknr)

In [ ]:
import imp

In [ ]:
srpoints = imp.reload(srpoints)
srreport = imp.reload(srreport)

In [ ]:
report = srreport.Report(weeknr)
for row in report.tournaments.values():
    print(row[TI.NAME])

## 2.

In [ ]:
print(report.raw_points[(37396, 673777)])
print(report.points[(37396, 673777)])
print(report.raw_points[(37396, 707219)])
print(report.points[(37396, 707219)])

In [ ]:
report.team_points[(37396, 673777)]

In [ ]:
report.coach_points[(38102, 'Kalos86')]

For every performances of those tournaments calculate the Points according to ''Table 1'' and ''Table 2''.

In [ ]:
weeknr_points = {}
for row in tournaments:
    main_id = (row[TI.TOP_ID] if 0 < row[TI.TOP_ID] else row[TI.ID])
    results = srschedule.get_results(row[TI.ID])
    for teamId, progression in results.items():
        key = (main_id, teamId)
        pts = srpoints.calculate(row[TI.CLASS], progression, (teamId==row[TI.WINNER_ID]))
        is_ugly = srpoints.is_ugly(progression)
        weeknr_points.setdefault(key, []).append([row[TI.ID], progression, is_ugly, pts])

In [ ]:
points_of_tournaments = srpoints.points_of_tournaments(tournaments)

In [ ]:
[(k, v) for k, v in report.coach_teams_of_tournaments.items() if 1 < len(v)]

## 3.

Collect every coaches and teams who participated on those tournaments and note them their slot groups and their initial vacant slots according to the Report date and ''Table 3''

In [ ]:
teamIds = {t[1] for t in weeknr_points}
team_data = {teamId: fumbblapi.get__team(teamId) for teamId in teamIds}
base_team_slots = srslot.team_slots(weeknr)
team_slots = {teamId: copy.copy(base_team_slots) for teamId in teamIds}
sr_team_points = {teamId: 0 for teamId in teamIds}

In [ ]:
report.team_slots

In [ ]:
coach_names = set()
for teamId in teamIds:
    coach_name = team_data[teamId].get("coach", {}).get("name")
    if coach_name:
        coach_names.add(coach_name)
base_coach_slots = srslot.coach_slots(weeknr)
coach_slots = {coach_name: copy.copy(base_coach_slots) for coach_name in coach_names}
sr_coach_points = {coach_name: 0 for coach_name in coach_names}

In [ ]:
fallback_slot_group = srslot.fallback_slot_group(weeknr)
print(fallback_slot_group)
points_included = srslot.slot_points_included(weeknr)
print(points_included)

## 4.

Collect the performances of multiple applications (see ''Wasted Points'' section above) and share the least of the Points equally among them, fractions rounded down. Original Points should get replaced with the proportions.

In [ ]:
sorted({1:1, 2:2, 5:'a'})

In [ ]:
weeknr_coaches = {}
for main_id, teamId in weeknr_points:
    coach = team_data[teamId].get("coach", {}).get("name")
    key = (main_id, coach)
    weeknr_coaches.setdefault(key, set()).add(teamId)

In [ ]:
report.multiple_applications

In [ ]:
report.points

In [ ]:
multiples = {}
for (main_id, coach), teamIds in weeknr_coaches.items():
    multiples[(main_id, coach)] = m = {}
    main_row = srdata.data["tournament"][main_id]
    if 1 < len(teamIds):  # multiple application
        min_pts = min(sum(a[-1] for a in weeknr_points[(main_id, teamId)]) for teamId in teamIds)
        pts = min_pts // len(teamIds)
        print(f'{main_row[TI.NAME]} | {coach} | {min_pts}//{len(teamIds)} = {pts}')
        for teamId in teamIds:
            li = weeknr_points[(main_id, teamId)]
            m[teamId] = copy.copy(li)
            for tournamentId, progression, is_ugly, pts_ in li:
                row = srdata.data["tournament"][tournamentId]
                team_name = team_data[teamId].get("name", "")
                print(f'{row[TI.NAME]} | {team_name} | {progression} | {"+-"[is_ugly]} | {pts_}')
                print(tournamenturlfs.format(groupId=row[TI.GROUP_ID], tournamentId=row[TI.ID]))
            print('before: ', weeknr_points[(main_id, teamId)])
            weeknr_points[(main_id, teamId)] = [[main_id, 'M', any(a[-2] for a in weeknr_points[(main_id, teamId)]), pts]]
            print('after: ', weeknr_points[(main_id, teamId)])
        print()

In [ ]:
team_points = {}
for (main_id, teamId), li in weeknr_points.items():
    is_ugly = any(a[-2] for a in li)
    pts = sum(a[-1] for a in li)
    team_points[(main_id, teamId)] = [is_ugly, pts]

In [ ]:
coach_points = {}
for (main_id, coach_name), teamIds in weeknr_coaches.items():
    is_ugly = any(team_points[(main_id, teamId)][-2] for teamId in teamIds)
    pts = sum(team_points[(main_id, teamId)][-1] for teamId in teamIds)
    coach_points[(main_id, coach_name)] = [is_ugly, pts]

## 5.

Collect the performances with dropouts/forfeits/concessions and sort them by points increasing, then enter date earliest first. Regardless of tournament rank, these points take vacant __R__ slots from the coaches and teams. If there are no more __R__ slot to hold the result, that performance got ''Wasted''.

In [ ]:
ugly_team_points = sorted(
        [pts, srdata.data["tournament"][main_id][TI.ENTER_WEEKNR], teamId]
        for (main_id, teamId), (is_ugly, pts) in team_points.items() 
        if is_ugly
)

In [ ]:
for pts, enter_weeknr, teamId in ugly_team_points:
    slot_group = "R"
    vacant_slot = team_slots[teamId][slot_group]
    while not vacant_slot:
        slot_group = fallback_slot_group[slot_group]
        vacant_slot = team_slots[teamId][slot_group]
    team_slots[teamId][slot_group] -= 1
    if points_included[slot_group]:
        sr_team_points[teamId] += pts

In [ ]:
ugly_coach_points = sorted(
        [pts, srdata.data["tournament"][main_id][TI.ENTER_WEEKNR], coach_name]
        for (main_id, coach_name), (is_ugly, pts) in coach_points.items() 
        if is_ugly
)

In [ ]:
for pts, enter_weeknr, coach_name in ugly_coach_points:
    slot_group = "R"
    vacant_slot = coach_slots[coach_name][slot_group]
    while not vacant_slot:
        slot_group = fallback_slot_group[slot_group]
        vacant_slot = coach_slots[coach_name][slot_group]
    coach_slots[coach_name][slot_group] -= 1
    if points_included[slot_group]:
        sr_coach_points[coach_name] += pts

## 6.

Sort the remaining performances by points decreasing then enter date earliest first. Get the ''First Slot Group'' value of the tournament as it is listed in the [Tournaments|SR-Tournaments] page and check for vacant slot for their coach and team in the obtained Slot Group. If that group is full, repeat this process for the next slot group. The order is __FC__ --> __MA__ --> __R__ --> ''Wasted''.

In [ ]:
nice_team_points = sorted(
        [-pts, srdata.data["tournament"][main_id][TI.ENTER_WEEKNR], teamId, main_id]
        for (main_id, teamId), (is_ugly, pts) in team_points.items() 
        if not is_ugly
)
nice_team_points = [[-a[0]] + a[1:] for a in nice_team_points]

In [ ]:
for pts, enter_weeknr, teamId, main_id in nice_team_points:
    slot_group = srdata.data["tournament"][main_id][TI.FIRST_SLOT_GROUP]
    vacant_slot = team_slots[teamId][slot_group]
    while not vacant_slot:
        slot_group = fallback_slot_group[slot_group]
        vacant_slot = team_slots[teamId][slot_group]
    team_slots[teamId][slot_group] -= 1
    if points_included[slot_group]:
        sr_team_points[teamId] += pts

In [ ]:
nice_coach_points = sorted(
        [-pts, srdata.data["tournament"][main_id][TI.ENTER_WEEKNR], coach_name, main_id]
        for (main_id, coach_name), (is_ugly, pts) in coach_points.items() 
        if not is_ugly
)
nice_coach_points = [[-a[0]] + a[1:] for a in nice_coach_points]

In [ ]:
for pts, enter_weeknr, coach_name, main_id in nice_coach_points:
    slot_group = srdata.data["tournament"][main_id][TI.FIRST_SLOT_GROUP]
    vacant_slot = coach_slots[coach_name][slot_group]
    while not vacant_slot:
        slot_group = fallback_slot_group[slot_group]
        vacant_slot = coach_slots[coach_name][slot_group]
    coach_slots[coach_name][slot_group] -= 1
    if points_included[slot_group]:
        sr_coach_points[coach_name] += pts

## 7.

Sum the points of each coaches and teams which were not ''Wasted'' and sort them. These are the ''SR Points''. Points of coaches and teams are published separately (Coach Rankings and Team Rankings).

In [ ]:
sorted(([pts, teamId, team_data[teamId].get("name", "")] for teamId, pts in sr_team_points.items()), reverse=True)[:25]

In [ ]:
sorted(([pts, coach_name] for coach_name, pts in sr_coach_points.items()), reverse=True)[:25]

In [ ]:
srtime.weeknr_firstdate(744)

In [ ]:
import srroster
srroster.name(46, 614)

In [ ]:
team_data[764724]

In [ ]:
fumbblapi.get__team(166748) == json.loads(empty)

In [ ]:
empty = '{"players": []}'
import json
json.loads(empty)